In [10]:
import pandas as pd
from pathlib import Path


def excel_diff(path_OLD, path_NEW):

    df_OLD = pd.read_excel(path_OLD).fillna(0)
    df_NEW = pd.read_excel(path_NEW).fillna(0)

    # Perform Diff
    dfDiff = df_OLD.copy()
    for row in range(dfDiff.shape[0]):
        for col in range(dfDiff.shape[1]):
            value_OLD = df_OLD.iloc[row,col]
            try:
                value_NEW = df_NEW.iloc[row,col]
                if value_OLD==value_NEW:
                    dfDiff.iloc[row,col] = df_NEW.iloc[row,col]
                else:
                    dfDiff.iloc[row,col] = ('{}-->{}').format(value_OLD,value_NEW)
            except:
                dfDiff.iloc[row,col] = ('{}-->{}').format(value_OLD, 'NaN')

    # Save output and format
    fname = '{} vs {}.xlsx'.format(path_OLD.stem,path_NEW.stem)
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')

    dfDiff.to_excel(writer, sheet_name='DIFF', index=False)
    df_NEW.to_excel(writer, sheet_name=path_NEW.stem, index=False)
    df_OLD.to_excel(writer, sheet_name=path_OLD.stem, index=False)

    # get xlsxwriter objects
    workbook  = writer.book
    worksheet = writer.sheets['DIFF']
    worksheet.hide_gridlines(2)

    # define formats
    date_fmt = workbook.add_format({'align': 'center', 'num_format': 'yyyy-mm-dd'})
    center_fmt = workbook.add_format({'align': 'center'})
    number_fmt = workbook.add_format({'align': 'center', 'num_format': '#,##0.00'})
    cur_fmt = workbook.add_format({'align': 'center', 'num_format': '$#,##0.00'})
    perc_fmt = workbook.add_format({'align': 'center', 'num_format': '0%'})
    grey_fmt = workbook.add_format({'font_color': '#E0E0E0'})
    highlight_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})

    # set column width and format over columns
    # worksheet.set_column('J:AX', 5, number_fmt)

    # set format over range
    ## highlight changed cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value':'-->',
                                            'format': highlight_fmt})
    ## highlight unchanged cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'not containing',
                                            'value':'-->',
                                            'format': grey_fmt})
    
    # save
    writer.save()
    print('Done.')

In [11]:
path_OLD=Path('Query_Net.xlsx')
path_NEW = Path('Query_Web.xlsx')
df_OLD = pd.read_excel(path_OLD).fillna(0)
df_NEW = pd.read_excel(path_NEW).fillna(0)
excel_diff(path_OLD, path_NEW)

Done.


In [12]:
import pandas as pd
from pathlib import Path

def excel_diff(path_OLD, path_NEW, index_col):

    df_OLD = pd.read_excel(path_OLD, index_col=index_col).fillna(0)
    df_NEW = pd.read_excel(path_NEW, index_col=index_col).fillna(0)
    print(path_OLD,path_OLD.stem)
        
    # Perform Diff
    dfDiff = df_NEW.copy()
    droppedRows = []
    newRows = []
    diffRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    
    for row in dfDiff.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]

                value_NEW = df_NEW.loc[row,col]
                if value_OLD==value_NEW:
                    dfDiff.loc[row,col] = df_NEW.loc[row,col]
                else:
                    dfDiff.loc[row,col] = ('{}-->{}').format(value_OLD,value_NEW)
                    diffRows.append(row)
        else:
            newRows.append(row)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            dfDiff = dfDiff.append(df_OLD.loc[row,:])

    dfDiff = dfDiff.sort_index().fillna('')
    #print(dfDiff)
    print('\nNew Rows:     {}'.format(newRows))
    print('Dropped Rows: {}'.format(droppedRows))

    # Save output and format
    fname = '{} vs {}.xlsx'.format(path_OLD.stem,path_NEW.stem)
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')

    dfDiff.to_excel(writer, sheet_name='DIFF', index=True)
    df_NEW.to_excel(writer, sheet_name=path_NEW.stem, index=True)
    df_OLD.to_excel(writer, sheet_name=path_OLD.stem, index=True)

    diffRows = list(set(diffRows+newRows+droppedRows))
    df_Changes = dfDiff.loc[diffRows,:]
    print('\nChanges:\n{}'.format(df_Changes))

    # get xlsxwriter objects
    workbook  = writer.book
    worksheet = writer.sheets['DIFF']
    worksheet.hide_gridlines(2)
    worksheet.set_default_row(15)

    # define formats
    #date_fmt = workbook.add_format({'align': 'center', 'num_format': 'yyyy-mm-dd'})
    center_fmt = workbook.add_format({'align': 'center'})
    number_fmt = workbook.add_format({'align': 'center', 'num_format': '#,##0.00'})
    cur_fmt = workbook.add_format({'align': 'center', 'num_format': '$#,##0.00'})
    perc_fmt = workbook.add_format({'align': 'center', 'num_format': '0%'})
    
    #grey_fmt = workbook.add_format({'font_color': '#E0E0E0', 'diag_type': 2,'diag_color': 'red'})
    grey_fmt = workbook.add_format({'font_color': '#E0E0E0'})
    #drop_fmt = workbook.add_format({'font_color': 'red', 'bold':True})   
    drop_fmt = workbook.add_format({'font_color': 'white', 'bg_color': 'red'})
    #new_fmt = workbook.add_format({'font_color': '#32CD32','bold':True})
    new_fmt = workbook.add_format({'font_color': 'white','bg_color': 'green'})
    highlight_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})

    # set format over range
    ## highlight changed cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value':'-->',
                                            'format': highlight_fmt})

#     ## highlight unchanged cells
#     worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
#                                             'criteria': 'not containing',
#                                             'value':'-->',
#                                             'format': grey_fmt})
    # highlight new/changed rows
    #for row in range(dfDiff.shape[0]):
    for i,row in enumerate(dfDiff.index):
        #print(row, newRows)
        if row in newRows:
            print('new row', row)
            worksheet.set_row(i+1, 15, new_fmt)
        if row in droppedRows:
            print('dropped rows', row)
            worksheet.set_row(i+1, 15, drop_fmt)

    # save
    writer.save()
    print('\nDone.\n')

In [13]:
path_OLD=Path('Query_Net.xlsx')
path_NEW = Path('Query_Web.xlsx')
df_OLD = pd.read_excel(path_OLD).fillna(0)
df_NEW = pd.read_excel(path_NEW).fillna(0)
index_col = 'ID'

In [14]:
 excel_diff(path_OLD, path_NEW,index_col)

Query_Net.xlsx Query_Net

New Rows:     ['DTRT20181  2018-08-31 00:00:00']
Dropped Rows: ['DTRT20181  2018-07-31 00:00:00']

Changes:
                                  TRUST_ID  PCD_ACT_D  Count  S_InvPrinBal  \
ID                                                                           
DTRT20181  2018-03-31 00:00:00  DTRT20181  2018-03-31   9703  1.066553e+09   
DTRT20181  2018-07-31 00:00:00  DTRT20181  2018-07-31   9112  9.080982e+08   
DTRT20181  2018-08-31 00:00:00  DTRT20181  2018-08-31   8112  8.080982e+08   

                                             S_InvBegPrinBal  
ID                                                            
DTRT20181  2018-03-31 00:00:00  1150009292.7-->1150009292.75  
DTRT20181  2018-07-31 00:00:00                   9.45698e+08  
DTRT20181  2018-08-31 00:00:00                   8.45698e+08  
dropped rows DTRT20181  2018-07-31 00:00:00
new row DTRT20181  2018-08-31 00:00:00

Done.

